# Ensemble Learning

## Initial Imports

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [7]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [8]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Encode CSV File

## Split the Data into Training and Testing

In [29]:
# Create our features
X = df.drop(columns=['loan_status'])
X = pd.get_dummies(X)
# Create our target
y = ensemble_df['loan_status']

In [30]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [31]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [32]:
# Import from sklearn.model_selection import train_test_split for splittling x and y train and test values. 
from sklearn.model_selection import train_test_split
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [33]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler()

In [34]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = StandardScaler()
X_scaler = X_scaler.fit(X_train)

In [35]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
39692,10000.0,0.1131,328.86,49000.0,24.54,0.0,0.0,10.0,1.0,6580.0,...,0,1,0,1,0,1,1,0,1,1
37830,21000.0,0.1072,684.74,57400.0,34.52,0.0,0.0,12.0,0.0,18688.0,...,0,1,0,1,0,1,1,0,1,1
64531,35000.0,0.0881,1109.90,180000.0,3.01,0.0,2.0,7.0,0.0,836.0,...,0,1,0,1,0,1,1,0,1,1
28730,20000.0,0.0819,628.49,80000.0,26.51,0.0,0.0,15.0,0.0,2615.0,...,0,1,0,1,0,1,1,0,1,1
35102,12600.0,0.1356,427.96,71000.0,22.38,1.0,2.0,7.0,0.0,9073.0,...,0,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18872,35000.0,0.0881,1109.90,400000.0,13.79,0.0,1.0,17.0,1.0,7810.0,...,0,1,0,1,1,0,1,0,1,1
6415,16800.0,0.0881,532.76,31000.0,20.48,0.0,0.0,3.0,0.0,1867.0,...,1,1,0,1,0,1,1,0,1,1
12632,25000.0,0.1102,818.71,90000.0,15.27,1.0,0.0,18.0,0.0,26106.0,...,0,1,0,1,1,0,1,0,1,1
24022,14000.0,0.1072,456.49,180000.0,12.35,2.0,0.0,10.0,0.0,3601.0,...,0,1,0,1,0,1,1,0,1,1


## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [36]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators = 100, random_state = 10)
brfc.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=10)

In [38]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.80311005555757

In [39]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   61,    26],
       [ 1625, 15493]], dtype=int64)

In [40]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.70      0.91      0.07      0.80      0.62        87
   low_risk       1.00      0.91      0.70      0.95      0.80      0.65     17118

avg / total       0.99      0.90      0.70      0.94      0.80      0.65     17205



In [43]:
# List the features sorted in descending order by feature importance
importance = brfc.feature_importances_
features = np.argsort(importance)[::-1]
for f in range(X.shape[1]):
    print(f"{X.columns.values[features[f]]}: ({importance[features]})")

total_rec_prncp: ([7.48317846e-02 6.60039677e-02 6.43366568e-02 6.13438679e-02
 5.23550630e-02 2.26537989e-02 1.95788954e-02 1.83313031e-02
 1.82893920e-02 1.75723047e-02 1.69956245e-02 1.59490406e-02
 1.59381387e-02 1.56847608e-02 1.51976865e-02 1.51611649e-02
 1.49258881e-02 1.47280471e-02 1.47161668e-02 1.39880579e-02
 1.38212135e-02 1.33235667e-02 1.30594963e-02 1.29249358e-02
 1.24036519e-02 1.21180366e-02 1.12999080e-02 1.12742873e-02
 1.11556395e-02 1.09956301e-02 1.07076548e-02 1.06409384e-02
 1.05679540e-02 1.04794505e-02 1.04724590e-02 1.02111489e-02
 1.01806802e-02 9.59113323e-03 9.52607425e-03 9.50723034e-03
 9.48837458e-03 9.45943102e-03 9.31649336e-03 9.17059150e-03
 9.14860809e-03 9.14744667e-03 8.48945658e-03 7.93205229e-03
 7.50612088e-03 7.48855543e-03 7.02865660e-03 6.62008182e-03
 6.57804905e-03 6.49634591e-03 5.98566974e-03 5.50528146e-03
 5.32163825e-03 5.30187026e-03 5.26344774e-03 5.20848430e-03
 5.05967065e-03 5.01469292e-03 4.66178432e-03 4.46742751e-03
 4.430

### Easy Ensemble Classifier

In [47]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
easy_ensemble = EasyEnsembleClassifier(n_estimators = 100, random_state = 10)
easy_ensemble.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=10)

In [48]:
# Calculated the balanced accuracy score
y_pred = easy_ensemble.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9328978167768551

In [49]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   80,     7],
       [  920, 16198]], dtype=int64)

In [50]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.92      0.95      0.15      0.93      0.87        87
   low_risk       1.00      0.95      0.92      0.97      0.93      0.87     17118

avg / total       0.99      0.95      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    - EasyEnsembleClassifier

2. Which model had the best recall score?

    - EasyEnsembleClassifier

3. Which model had the best geometric mean score?

    - EasyEnsembleClassifier

4. What are the top three features?

    - total_rec_prncp, total_rec_int, total_rec_inv
    - The fourth and fifth top features were last_pymnt_amnt, and total_pymnt